# Перемешанные фотографии

Возьмите [здесь](https://www.kaggle.com/datasets/amitsharma11jan/caltech-101/data) данные) Дальше ковыряться будем с ними :)

Представим, что у вас они теперь лежат в папке, которая называется **101_ObjectCategories**

In [1]:
import os
import hashlib
import shutil
import numpy as np

In [2]:
path = '.\\101_ObjectCategories'

In [3]:
#Ищем картинки
extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG']
def get_file_list(root_dir):
    file_list = []
    counter = 1
    for root, directories, filenames in os.walk(root_dir):
        for filename in filenames:
            if any(ext in filename for ext in extensions):
                file_list.append(os.path.join(root, filename))
                counter += 1
    return file_list

In [4]:
filenames = sorted(get_file_list(path))

Выберите 2000 случайных фотографий, перемешайте их названия, захешировав. Например так:

In [5]:
np.random.seed(52)
random2000 = np.random.choice(filenames, 2000)

def random_hash_name(filename):
    return '.\\Renamed\\' + hashlib.md5((filename+str(np.random.random())).encode('utf-8')).hexdigest()+'.jpg'

In [6]:
random_hash_name(random2000[0])

'.\\Renamed\\951e11facc8a0d7aa22ff63f832fccfc.jpg'

Пересохраните выбранные 2000 фотографий в соседнюю папку с новыми именами. а теперь задание:
    
1) Используя знания об ANN или любой другой удобный вам подход, восстановите соответствие между названием исходной фотографии и переименованной

2) Проверьте качество восстановления

3) С каким качеством получится восстановить соответствие, если каждую фотографию после запуска процедуры немного "испортить", например, добавить в нее случайный "шум", но так чтобы картинку все еще можно было легко узнать, или запустить на ней SVD и выбрать столько главных компонент, чтобы картинка была хорошо узнаваема? 

4) Проверьте качество восстановления на зашумленных любым способом данных



In [7]:
old_new_filenames = {}

for file in random2000:
    old_new_filenames[file] = random_hash_name(file)

In [8]:
os.makedirs('Renamed', exist_ok=True)

for old_file, new_file in old_new_filenames.items():
    shutil.copy2(old_file, new_file)

In [9]:
from sklearn.neighbors import NearestNeighbors
import cv2

In [10]:
def extract_features(image_path, bins=(8, 8, 8)):
    image = cv2.imread(image_path)
    if image is None:
        return None
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([image], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

In [11]:
old_features = []
new_features = []
old_paths = []
new_paths = []

for old_path, new_path in old_new_filenames.items():
    old_feat = extract_features(old_path)
    new_feat = extract_features(new_path)
    if old_feat is not None and new_feat is not None:
        old_features.append(old_feat)
        new_features.append(new_feat)
        old_paths.append(old_path)
        new_paths.append(new_path)

old_features = np.array(old_features)
new_features = np.array(new_features)

nn_model = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(old_features)
distances, indices = nn_model.kneighbors(new_features)

recovered_mapping = {}
for new_idx, old_idx in enumerate(indices.flatten()):
    recovered_mapping[new_paths[new_idx]] = old_paths[old_idx]

correct_matches = sum(recovered_mapping[new] == old for old, new in old_new_filenames.items())
accuracy = correct_matches / len(old_new_filenames)
print(f"Accuracy of recovery: {accuracy:.2%}")

Accuracy of recovery: 100.00%


Ожидаемый результат, так как у нас векторы признаков исходных и хэшированных фотографий одинаковые, и KNN легко с ними справляется

In [12]:
def add_noise(image, noise_level=150):
    noise = np.random.randint(-noise_level, noise_level, image.shape, dtype='int16')
    noisy_image = np.clip(image + noise, 0, 255)
    return noisy_image.astype('uint8')


def compress_svd(image, num_components=20):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    u, s, vt = np.linalg.svd(gray_image, full_matrices=False)
    compressed_img = np.dot(u[:, :num_components], np.dot(np.diag(s[:num_components]), vt[:num_components, :]))
    return np.clip(compressed_img, 0, 255).astype('uint8')


def extract_features_with_modifications(image_path, size=(32, 32), modify=None):
    image = cv2.imread(image_path)
    image = cv2.resize(image, size)
    if modify == 'noise':
        image = add_noise(image)
    elif modify == 'svd':
        image = compress_svd(image)
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    return image.flatten()


def evaluate_with_modifications(modify=None):
    modified_old_features = []
    modified_new_features = []
    
    for old_path, new_path in old_new_filenames.items():
        old_feat = extract_features_with_modifications(old_path)
        new_feat = extract_features_with_modifications(new_path, modify=modify)
        if old_feat is not None and new_feat is not None:
            modified_old_features.append(old_feat)
            modified_new_features.append(new_feat)
    
    nn_model = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(modified_old_features)
    distances, indices = nn_model.kneighbors(modified_new_features)
    
    recovered_mapping = {}
    for new_idx, old_idx in enumerate(indices.flatten()):
        recovered_mapping[new_paths[new_idx]] = old_paths[old_idx]
    
    correct_matches = sum(recovered_mapping[new] == old for old, new in old_new_filenames.items())
    accuracy = correct_matches / len(old_new_filenames)
    print(f"Accuracy with {modify or 'no modifications'}: {accuracy:.2%}")
    return accuracy

In [13]:
accuracy_with_noise = evaluate_with_modifications(modify='noise')
accuracy_with_svd = evaluate_with_modifications(modify='svd')

Accuracy with noise: 99.83%
Accuracy with svd: 97.14%


Видим, что при стремительнм увеличении шума accuracy уменьшается не так быстро. С сингулярным разложением другая история - accuracy сильно уменьшается при небольшом уменьшении количества главных компонент.  
В нашем случае, при использовании SVD с 20 главными компонентами мы всё равно отлично восстанавливаем схожесть между исходными и хэшированными фотографиями.

# Реализация Любой версии LogLog :)

Некоторое время назад мы обсуждали с вами реализацию алгоритмов семейства LogLog, где для расчета кардинальности множества подсчитывалось количество ведущих нулей в hash'е от элемента, а дальше на основании максимального кол-ва нулей, которое мы в множестве увидели, подсчитывалось количество элементов.

Далее товарищ Флажоле придумал это делать несколько раз и усреднять.

Как ведущие нули посчитать, можно [нагуглить](https://stackoverflow.com/questions/71888646/counting-the-number-of-leading-zero-bits-in-a-sha256-encrpytion)

Задание: реализуйте любой из алгоритмов семейства LogLog (пусть даже самого простого Флажоле-Мартена) и оцените его уровень ошибки на случайно сгенерированном множестве из случайных строк с повторениями 

In [14]:
import hashlib
import numpy as np
import string

In [15]:
def hash_function(item):
    return int(hashlib.md5(item.encode('utf-8')).hexdigest(), 16)


def count_leading_zeros(x, max_bits):
    binary = bin(x)[2:].zfill(max_bits)
    return binary.find('1')


def flajolet_martin(stream, num_hash_functions=10):
    max_bits = 128
    max_zeros = np.zeros(num_hash_functions, dtype=int)

    for item in stream:
        for i in range(num_hash_functions):
            combined_item = f"{item}_{i}"
            hashed_value = hash_function(combined_item)
            leading_zeros = count_leading_zeros(hashed_value, max_bits)
            max_zeros[i] = max(max_zeros[i], leading_zeros)

    estimates = 2 ** max_zeros
    return np.mean(estimates)


def generate_random_strings(size, string_length=10):
    return [''.join(np.random.choice(list(string.ascii_letters), size=string_length)) for _ in range(size)]


def evaluate_error(true_cardinality, estimated_cardinality):
    return abs(estimated_cardinality - true_cardinality) / true_cardinality

In [16]:
stream_size = 1000
unique_elements = 500
num_hash_functions = 10

np.random.seed(9)
unique_stream = generate_random_strings(unique_elements)
random_stream = unique_stream + list(np.random.choice(unique_stream, size=stream_size - unique_elements))
np.random.shuffle(random_stream)

true_cardinality = len(set(random_stream))
estimated_cardinality = flajolet_martin(random_stream, num_hash_functions=num_hash_functions)
error = evaluate_error(true_cardinality, estimated_cardinality)

print(f"Посчитанная кардинальность: {estimated_cardinality}")
print(f"Истинная кардинальность:    {true_cardinality}")
print(f"Относительная ошибка:       {error:.3f}")

Посчитанная кардинальность: 576.0
Истинная кардинальность:    500
Относительная ошибка:       0.152
